# Try arrow vs numpy on some ops?

In [1]:
import pandas as pd
import numpy as np
import polars as pl
#%load_ext autoreload
#%autoreload 2

import datetime, os
display(f"Pandas {pd.__version__}")
display(f'Running: {datetime.datetime.now().strftime("%Y%m%d %H:%M:%S")}, env {os.getenv("CONDA_DEFAULT_ENV")}')


%load_ext ipython_memory_usage
%imu_start

'Pandas 2.1.3'

'Running: 20231206 16:03:12, env pydataglobal2023'

Enabling IPython Memory Usage, use %imu_start to begin, %imu_stop to end


'IPython Memory Usage started'

In [1] used 0.2 MiB RAM in 0.11s (system mean cpu 0%, single max cpu 0%), peaked 0.0 MiB above final usage, current RAM usage now 149.3 MiB


In [17]:
arr = np.ones((500_000_000, 2), dtype=np.float64)
f"{(arr.nbytes)/1000000:,} MB"

'8,000.0 MB'

In [17] used -2802.0 MiB RAM in 3.28s (system mean cpu 15%, single max cpu 100%), peaked 7684.1 MiB above final usage, current RAM usage now 42263.3 MiB


In [8]:
dfpla = pl.DataFrame(arr)
dfpla.head()
#  7632.1 MiB RAM in 3s , so polars needs to copy arr into arrow format ??
# 7GB 3s float64

column_0,column_1
f64,f64
1.0,1.0
1.0,1.0
1.0,1.0
1.0,1.0
1.0,1.0


In [8] used 7629.5 MiB RAM in 3.44s (system mean cpu 14%, single max cpu 100%), peaked 0.0 MiB above final usage, current RAM usage now 38300.2 MiB


In [21]:
dfpdn2 = dfpla.to_pandas() # costs 5GB and 3s

In [21] used 7629.0 MiB RAM in 2.19s (system mean cpu 18%, single max cpu 100%), peaked 0.0 MiB above final usage, current RAM usage now 42264.5 MiB


In [22]:
dfpdn2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500000000 entries, 0 to 499999999
Data columns (total 2 columns):
 #   Column    Dtype  
---  ------    -----  
 0   column_0  float64
 1   column_1  float64
dtypes: float64(2)
memory usage: 7.5 GB
In [22] used 0.0 MiB RAM in 0.10s (system mean cpu 9%, single max cpu 44%), peaked 0.0 MiB above final usage, current RAM usage now 42264.5 MiB


In [20]:
dfpda2 = dfpla.to_pandas(use_pyarrow_extension_array=True)
dfpda2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500000000 entries, 0 to 499999999
Data columns (total 2 columns):
 #   Column    Dtype          
---  ------    -----          
 0   column_0  double[pyarrow]
 1   column_1  double[pyarrow]
dtypes: double[pyarrow](2)
memory usage: 7.5 GB
In [20] used -7628.5 MiB RAM in 0.42s (system mean cpu 5%, single max cpu 7%), peaked 7628.6 MiB above final usage, current RAM usage now 34635.5 MiB


In [14]:
f"{arr[:, 0].data.nbytes:,} bytes" # 4GB

'8,000,000,000 bytes'

In [14] used 0.0 MiB RAM in 0.10s (system mean cpu 9%, single max cpu 18%), peaked 0.0 MiB above final usage, current RAM usage now 49755.6 MiB


In [10]:
import pyarrow as pa
#pa.array(arr) # only 1d arrays!
arrpa = pa.array(arr[:, 0]) # only 1d arrays!  used 3814.8 MiB RAM in 2.28s , float and int same
arrpa[:3]

[
  1,
  1,
  1
]

In [10] used 3814.9 MiB RAM in 2.07s (system mean cpu 15%, single max cpu 100%), peaked 0.0 MiB above final usage, current RAM usage now 49747.7 MiB


In [18]:
arrpa.to_numpy()

array([1., 1., 1., ..., 1., 1., 1.])

In [18] used 0.1 MiB RAM in 0.11s (system mean cpu 5%, single max cpu 9%), peaked 0.0 MiB above final usage, current RAM usage now 42263.4 MiB
